In [1]:
ls

speechRecognitionv1.ipynb


# Nmedia/ction

Installation of Pydub for Audio File Manipulation and Analysis of audio files

In [2]:
!pip install pydub torchaudio torch comet-ml
!pip install torchaudio torch comet-ml

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [3]:
import os
import pickle
import random
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
import numpy as np
from pydub import AudioSegment, effects
import matplotlib.pyplot as plt

In [4]:
def _levenshtein_distance(ref, hyp):
    """Levenshtein distance is a string metric for measuring the difference
    between two sequences. Informally, the levenshtein disctance is defined as
    the minimum number of single-character edits (substitutions, insertions or
    deletions) required to change one word into the other. We can naturally
    extend the edits to word level when calculate levenshtein disctance for
    two sentences.
    """
    m = len(ref)
    n = len(hyp)

    # special case
    if ref == hyp:
        return 0
    if m == 0:
        return n
    if n == 0:
        return m

    if m < n:
        ref, hyp = hyp, ref
        m, n = n, m

    # use O(min(m, n)) space
    distance = np.zeros((2, n + 1), dtype=np.int32)

    # initialize distance matrix
    for j in range(0,n + 1):
        distance[0][j] = j

    # calculate levenshtein distance
    for i in range(1, m + 1):
        prev_row_idx = (i - 1) % 2
        cur_row_idx = i % 2
        distance[cur_row_idx][0] = i
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                distance[cur_row_idx][j] = distance[prev_row_idx][j - 1]
            else:
                s_num = distance[prev_row_idx][j - 1] + 1
                i_num = distance[cur_row_idx][j - 1] + 1
                d_num = distance[prev_row_idx][j] + 1
                distance[cur_row_idx][j] = min(s_num, i_num, d_num)

    return distance[m % 2][n]

In [5]:
def word_errors(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in word-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Levenshtein distance and word number of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    ref_words = reference.split(delimiter)
    hyp_words = hypothesis.split(delimiter)

    edit_distance = _levenshtein_distance(ref_words, hyp_words)
    return float(edit_distance), len(ref_words)

In [6]:
def char_errors(reference, hypothesis, ignore_case=False, remove_space=False):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in char-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Levenshtein distance and length of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = ' '
    if remove_space == True:
        join_char = ''

    reference = join_char.join(filter(None, reference.split(' ')))
    hypothesis = join_char.join(filter(None, hypothesis.split(' ')))

    edit_distance = _levenshtein_distance(reference, hypothesis)
    return float(edit_distance), len(reference)

In [7]:
def wer(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Calculate word error rate (WER). WER compares reference text and
    hypothesis text in word-level. WER is defined as:
    .. math::
        WER = (Sw + Dw + Iw) / Nw
    where
    .. code-block:: text
        Sw is the number of words subsituted,
        Dw is the number of words deleted,
        Iw is the number of words inserted,
        Nw is the number of words in the reference
    We can use levenshtein distance to calculate WER. Please draw an attention
    that empty items will be removed when splitting sentences by delimiter.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Word error rate.
    :rtype: float
    :raises ValueError: If word number of reference is zero.
    """
    edit_distance, ref_len = word_errors(reference, hypothesis, ignore_case,
                                         delimiter)

    if ref_len == 0:
        raise ValueError("Reference's word number should be greater than 0.")

    wer = float(edit_distance) / ref_len
    return wer

In [8]:
def cer(reference, hypothesis, ignore_case=False, remove_space=False):
    """Calculate charactor error rate (CER). CER compares reference text and
    hypothesis text in char-level. CER is defined as:
    .. math::
        CER = (Sc + Dc + Ic) / Nc
    where
    .. code-block:: text
        Sc is the number of characters substituted,
        Dc is the number of characters deleted,
        Ic is the number of characters inserted
        Nc is the number of characters in the reference
    We can use levenshtein distance to calculate CER. Chinese input should be
    encoded to unicode. Please draw an attention that the leading and tailing
    space characters will be truncated and multiple consecutive space
    characters in a sentence will be replaced by one space character.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Character error rate.
    :rtype: float
    :raises ValueError: If the reference length is zero.
    """
    edit_distance, ref_len = char_errors(reference, hypothesis, ignore_case,
                                         remove_space)

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer

In [9]:
class TextTransform:
    """Maps characters to integers and vice versa"""
    def __init__(self):
        char_map_str = """
        ' 0
        <SPACE> 1
        < 1
        . 1
        a 2
        b 3
        c 4
        d 5
        e 6
        f 7
        g 8
        h 9
        i 10
        j 11
        k 12
        l 13
        m 14
        n 15
        o 16
        p 17
        q 18
        r 19
        s 20
        t 21
        u 22
        v 23
        w 24
        x 25
        y 26
        z 27
        """
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[1] = ' '

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text.replace("\n", ""):
            if c == ' ':
                ch = self.char_map['<SPACE>']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('<SPACE>', ' ')

In [10]:
train_audio_transforms = nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
    torchaudio.transforms.TimeMasking(time_mask_param=100)
)

/home/paperspace/.local/lib/python3.9/site-packages/torchaudio/functional/functional.py:539: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [11]:
valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

In [12]:
text_transform = TextTransform()

In [13]:
def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        elif data_type == 'valid':
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            raise Exception('data_type should be train or valid')
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths

In [14]:
def data_processing_2(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, utterance) in data:
        if data_type == 'train':
            spec = train_audio_transforms(torch.from_numpy(waveform).float()).squeeze(0).transpose(0, 1)
        elif data_type == 'valid':
            spec = valid_audio_transforms(torch.from_numpy(waveform).float()).squeeze(0).transpose(0, 1)
        else:
            raise Exception('data_type should be train or valid')

        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths

In [15]:
def GreedyDecoder(output, labels, label_lengths, blank_label=28, collapse_repeated=True):
	arg_maxes = torch.argmax(output, dim=2)
	decodes = []
	targets = []
	for i, args in enumerate(arg_maxes):
		decode = []
		targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
		for j, index in enumerate(args):
			if index != blank_label:
				if collapse_repeated and j != 0 and index == args[j -1]:
					continue
				decode.append(index.item())
		decodes.append(text_transform.int_to_text(decode))
	return decodes, targets

# The Model
Bases on Deep Speech 2 with some modifications

**CNN Layer Norm**

**Residual CNN**

**Bidirectioal GRU**



In [16]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time)

In [17]:
class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)

In [18]:
class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x

In [19]:
class SpeechRecognitionModel(nn.Module):
    
    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[
            ResidualCNN(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*32, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x

In [20]:
class IterMeter(object):
    """keeps track of total iterations"""
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val

In [21]:
def train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter):
    model.train()
    data_len = len(train_loader.dataset)
    train_loss = 0

    for batch_idx, _data in enumerate(train_loader):
            spectrograms, labels, input_lengths, label_lengths = _data 
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            optimizer.zero_grad()

            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1) # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            loss.backward()

            optimizer.step()
            scheduler.step()
            iter_meter.step()
            train_loss += loss.item()/ len(train_loader)
            
            

           # if batch_idx % 1 == 0 or data_len == ((batch_idx + 1) * 5) :
            #    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
             #       epoch, (batch_idx + 1) * len(spectrograms), data_len,
              #      100. * batch_idx / len(train_loader), loss.item()))
                
    print('Train Epoch: {}: Average loss: {:.4f}'.format(epoch,train_loss))

In [22]:
def test(model, device, test_loader, criterion, epoch, iter_meter):
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []

    with torch.no_grad():
            for i, _data in enumerate(test_loader):
                spectrograms, labels, input_lengths, label_lengths = _data 
                spectrograms, labels = spectrograms.to(device), labels.to(device)

                output = model(spectrograms)  # (batch, time, n_class)
                output = F.log_softmax(output, dim=2)
                output = output.transpose(0, 1) # (time, batch, n_class)

                loss = criterion(output, labels, input_lengths, label_lengths)
                test_loss += loss.item() / len(test_loader)

                decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
                for j in range(len(decoded_preds)):
                    test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                    test_wer.append(wer(decoded_targets[j], decoded_preds[j]))


    avg_cer = sum(test_cer)/len(test_cer)
    avg_wer = sum(test_wer)/len(test_wer)

    print('Test set: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}'.format(test_loss, avg_cer, avg_wer))

In [23]:
def avg_wer(wer_scores, combined_ref_len):
    return float(sum(wer_scores)) / float(combined_ref_len)

In [24]:
ls 

speechRecognitionv1.ipynb


In [25]:
ls /media/kremon-storage/dataset-01

'Batch 1 May Kelly 0.pickle'*  'Batch 1 May Kelly 7.pickle'*
'Batch 1 May Kelly 1.pickle'*  'Batch 1 May Kelly 8.pickle'*
'Batch 1 May Kelly 2.pickle'*  'Batch 1 May Kelly 9.pickle'*
'Batch 1 May Kelly 3.pickle'*  'Batch 2 May Kelly 0.pickle'*
'Batch 1 May Kelly 4.pickle'*  'Batch 2 May Kelly 1.pickle'*
'Batch 1 May Kelly 5.pickle'*  'Batch 2 May Kelly 2.pickle'*
'Batch 1 May Kelly 6.pickle'*  'Batch 2 May Kelly 3.pickle'*


In [26]:
#pickle_list = ['data_16kHz.pickle','Batch 1 2021 David Segmented pt1.pickle','Batch 1 2021 David Segmented pt3.pickle','Batch 1 2021 David Segmented pt4.pickle','Batch 1 2021 David Segmented pt5.pickle','Batch 1 2021 David Segmented pt7.pickle']
pickle_list = [
    'Batch 1 May Kelly 0.pickle',
    'Batch 1 May Kelly 1.pickle',
    'Batch 1 May Kelly 2.pickle',
    'Batch 1 May Kelly 3.pickle',
    'Batch 1 May Kelly 4.pickle',
    'Batch 1 May Kelly 5.pickle',
    'Batch 1 May Kelly 6.pickle',
    'Batch 1 May Kelly 7.pickle',
    'Batch 1 May Kelly 8.pickle',
    'Batch 1 May Kelly 9.pickle',
    'Batch 2 May Kelly 0.pickle',
    'Batch 2 May Kelly 1.pickle',
    'Batch 2 May Kelly 2.pickle',
    'Batch 2 May Kelly 3.pickle'
]
b = []
for filename in pickle_list:
    print("Working with: "+filename)
    with open('/media/kremon-storage/dataset-01/'+filename, 'rb') as handle:
        b = b + pickle.load(handle)
        
random.shuffle(b)

Working with: Batch 1 May Kelly 0.pickle
Working with: Batch 1 May Kelly 1.pickle
Working with: Batch 1 May Kelly 2.pickle
Working with: Batch 1 May Kelly 3.pickle
Working with: Batch 1 May Kelly 4.pickle
Working with: Batch 1 May Kelly 5.pickle
Working with: Batch 1 May Kelly 6.pickle
Working with: Batch 1 May Kelly 7.pickle
Working with: Batch 1 May Kelly 8.pickle
Working with: Batch 1 May Kelly 9.pickle
Working with: Batch 2 May Kelly 0.pickle
Working with: Batch 2 May Kelly 1.pickle
Working with: Batch 2 May Kelly 2.pickle
Working with: Batch 2 May Kelly 3.pickle


In [ ]:
2300

In [27]:
train_set = b[0:2000]
test_set = b[2000:2300]

In [28]:
len(b)

1400

In [29]:



kwargs = {'num_workers': 1, 'pin_memory': True} 

hparams = {
        "n_cnn_layers": 3,
        "n_rnn_layers": 5,
        "rnn_dim": 512,
        "n_class": 29,
        "n_feats": 128,
        "stride":2,
        "dropout": 0.1,
        "learning_rate": 0.01,
        "batch_size": 30,
        "epochs": 50
    }


train_2_loader = data.DataLoader(dataset=train_set,
                                batch_size=hparams["batch_size"],
                                shuffle=True,
                                collate_fn=lambda x: data_processing_2(x, 'train'),
                                **kwargs)


test_2_loader = data.DataLoader(dataset=test_set,
                                batch_size=5,
                                shuffle=False,
                                collate_fn=lambda x: data_processing_2(x, 'valid'),
                                **kwargs)

use_cuda = torch.cuda.is_available()

torch.manual_seed(7)

device = torch.device("cuda")





model = SpeechRecognitionModel(
            hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
            hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
            ).to(device)

In [30]:
print(model)

SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,),

In [31]:


          
#Test different optimizers 
#optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
#optimizer = optim.SGD(model.parameters(), lr = hparams['learning_rate'], momentum = 0.3)

optimizer = torch.optim.Adam(model.parameters(), lr=hparams['learning_rate'])
    

criterion = nn.CTCLoss(blank=28).to(device)


# Look into this learning rate scheduler seems odd?!
# anneal strategy????

scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], 
                                                steps_per_epoch=int(len(train_2_loader)),
                                                epochs=hparams['epochs'],
                                                anneal_strategy='linear')

epochs = hparams['epochs']
iter_meter = IterMeter()
for epoch in range(1, epochs + 1):
        train(model, device, train_2_loader, criterion, optimizer, scheduler, epoch, iter_meter)
        test(model, device, test_2_loader, criterion, epoch, iter_meter)

Train Epoch: 1: Average loss: 5.0366
Test set: Average loss: 3.0065, Average CER: 1.000000 Average WER: 0.9906
Train Epoch: 2: Average loss: 2.9087
Test set: Average loss: 2.9354, Average CER: 1.000000 Average WER: 0.9906
Train Epoch: 3: Average loss: 2.8693
Test set: Average loss: 2.9079, Average CER: 1.000000 Average WER: 0.9906
Train Epoch: 4: Average loss: 2.8547
Test set: Average loss: 2.8949, Average CER: 1.000000 Average WER: 0.9906
Train Epoch: 5: Average loss: 2.8511
Test set: Average loss: 2.9057, Average CER: 1.000000 Average WER: 0.9906
Train Epoch: 6: Average loss: 2.8625
Test set: Average loss: 2.9126, Average CER: 1.000000 Average WER: 0.9906
Train Epoch: 7: Average loss: 2.8397
Test set: Average loss: 2.8940, Average CER: 1.000000 Average WER: 0.9906
Train Epoch: 8: Average loss: 2.8480
Test set: Average loss: 3.2299, Average CER: 1.000000 Average WER: 0.9906
Train Epoch: 9: Average loss: 3.0719
Test set: Average loss: 2.9485, Average CER: 1.000000 Average WER: 0.9906
T

In [33]:
model.clear_param_store()

AttributeError: 'SpeechRecognitionModel' object has no attribute 'clear_param_store'

In [5]:
! pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=fb45c1d7e2db089f0410647524710807d225663a3750992df1e8e29b119abb4c
  Stored in directory: /root/.cache/pip/wheels/79/a7/0c/d53673ba405bcb572989c614f77c87833b03c3ca07d1f3e48e
Successfully built GPUtil


In [8]:
! pip install numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 62.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 44.4 MB/s eta 0:00:0000:0100:01


In [37]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [40]:
gpus = cuda.list_devices()

In [41]:
gpus

In [50]:
import gc
gc.collect()

2881

In [47]:
torch.cuda.empty_cache()

In [36]:
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

device = cuda.get_current_device()
device.reset()

In [43]:
gpus = cuda.list_devices()

In [42]:
device

device(type='cuda')

In [45]:
del test_2_loader

In [37]:
print(torch.cuda.memory_allocated())

11436754432


In [35]:
!nvidia-smi

Mon May 29 10:29:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0 Off |                  Off |
| 41%   48C    P8    16W / 140W |  15703MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [33]:
def memory_stats():
    print(torch.cuda.memory_allocated()/1024**2)
    print(torch.cuda.memory_cached()/1024**2)

In [35]:
torch.cuda.empty_cache()

In [38]:
!pip install numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 44.3 MB/s eta 0:00:0000:0100:01


In [39]:
from numba import cuda


<weakproxy at 0x7f0025f20a40 to Device at 0x7f002c0c59a0>

In [54]:
!nvtop

/bin/bash: nvtop: command not found


In [45]:
memory_stats()

90.60693359375
14136.0


In [55]:
! nvidia-smi --gpu-reset -i 0

Error occurred during reset of GPU 00000000:00:05.0: Unknown Error

1 device did not complete reset successfully, and may be in an unstable state. Please reboot your system.


In [49]:
torch.cuda.empty_cache()

RuntimeError: CUDA error: invalid argument
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [47]:
!nvidia-smi -r

Error occurred during reset of GPU 00000000:00:05.0: Unknown Error

1 device did not complete reset successfully, and may be in an unstable state. Please reboot your system.


In [48]:
torch.cuda.empty()

AttributeError: module 'torch.cuda' has no attribute 'empty'

In [70]:
print(torch.cuda.is_available())

True
